## TP6

Una empresa de transporte quiere monitorear la actividad de sus conductores durante el manejo Para ello se propone implementar
un detector de somnolencia en tiempo real tomando como datos de entrada frames provenientes de una cámara que apunta al rosto
del conductor

1. Implementar un detector Haar de caras (vista frontal)

2. En esa ROI detectar los ojos (Se puede usar otro detector Haar

3. Mediante alguna de las técnicas vistas en clase detectar si ambos ojos están abiertos o cerrados y mostrar el estado de la detección sobre el frame


In [3]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [4]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
eye_cascade = cv.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')

In [ ]:

cap = cv.VideoCapture(1)
selected_roi = False
show_track = True

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output.mp4', fourcc, 5, (int(cap.get(3)), int(cap.get(4))))

counter_no_eyes = 0
centers = list()
while(1):

    # 480 x 640
    ret ,frame = cap.read()

    if ret:
          
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        #gray = cv.equalizeHist(gray)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, 1.1, minNeighbors=5)#, minSize= (100,130),maxSize=(200,400))

        if len(faces) > 0:
            for (x,y,w,h) in faces:
                # Le dibujamos un rectángulo amarillo
                cv.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
                # Definimos las ROIs en la imagen gris y color
                roi_gray = gray[y:y+h, x:x+w] 
                roi_color = frame[y:y+h, x:x+w] 
                # Para cada rostro hallado le buscamos los ojos
                eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 8)
                # En los ojos hallados les dibujamos rectángulos
                for (ex,ey,ew,eh) in eyes:
                    cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                if len(eyes) == 0:
                    cv.putText(frame, 'Sin ojos', (20, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
                    #print('Ambos ojos cerrados')
                    cv.putText(frame, 'Ambos ojos cerrados', (20, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
                else:                        
                    cv.putText(frame, f'# Ojos abiertos={len(eyes)}', (20, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
                
         

        cv.imshow('Face Recognition', frame)
        out.write(frame)
        
        key = cv.waitKey(50) & 0xFF
            
        if key == ord("q"):
            break
    else:
        break

cv.destroyAllWindows()
out.release()
cap.release()